In [3]:
%pip install sacremoses sacrebleu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import gc
import torch
import re
import random
import sys
from collections import Counter
import unicodedata
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
from sklearn.model_selection import train_test_split

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, NllbTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import get_constant_schedule_with_warmup
from transformers.optimization import Adafactor

import sentencepiece as spm
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model

from torch.optim.lr_scheduler import SequentialLR, ExponentialLR, ConstantLR, LinearLR
from sacremoses import MosesPunctNormalizer

import json
import os
import shutil
from typing import List, Tuple

from transformers.models.nllb.tokenization_nllb import FAIRSEQ_LANGUAGE_CODES

def cleanup():
    """Try to free GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()
cleanup()

In [ ]:
# res = pd.read_csv('/kaggle/input/wiki-to-label/res.csv')
# tartu_trans = pd.read_csv('/kaggle/input/wiki-to-label/tartu_trans.csv',index_col=0)

In [ ]:
# trans_10k_all = pd.concat([res, tartu_trans],axis=1)
# trans_10k_all.to_csv('trans_10k_all.csv', index=False)

In [5]:
KAGGLE_INPUT = '/kaggle/input/'
MODEL_PATH = 'nllb-rus-kar'
KRL = KAGGLE_INPUT + 'karelian-data/'
DF_PATH = KRL + 'parallel_df_merged4.csv'
# MODEL_LOAD_PATH = KAGGLE_INPUT + '/m/taciturno/nllb-rus-kar/pytorch/ft-messy/1'
MODEL_SAVE_PATH = MODEL_PATH

In [6]:
df_corpus_labeled = pd.read_csv(DF_PATH)
df_train = df_corpus_labeled[df_corpus_labeled.split=='train'].copy() # 22692 items
df_dev = df_corpus_labeled[df_corpus_labeled.split=='dev'].copy()     # 500 items
df_test = df_corpus_labeled[df_corpus_labeled.split=='test'].copy()  

df_dev = df_dev.sample(frac=1).reset_index(drop=True)

In [ ]:
# df_train = trans_10k_all[['orig', 'rus']].copy()
# df_train.columns = ['kar', 'rus']

Preproc

In [ ]:
mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]

def get_non_printing_char_replacer(replace_by: str = " "):
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        # same as \p{C} in perl
        # see https://www.unicode.org/reports/tr44/#General_Category_Values
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    # replace 𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞 by Francesca
    clean = unicodedata.normalize("NFKC", clean)
    return clean

Tokenizer

In [ ]:
additional_special_tokens=sorted(FAIRSEQ_LANGUAGE_CODES + ['olo_Latn'])

In [ ]:
model_name = 'facebook/nllb-200-distilled-600M' # 'facebook/nllb-200-distilled-1.3B' #
NEW_SPM_NAME = '/kaggle/input/karelian-data/spm_nllb_karelian.model'
# loading the tokenizers
tokenizer_old = NllbTokenizer.from_pretrained(model_name)
tokenizer = NllbTokenizer.from_pretrained(model_name, vocab_file=NEW_SPM_NAME, additional_special_tokens=additional_special_tokens)
print(len(tokenizer_old), len(tokenizer)) # 256204, 268559
added_vocab = set(tokenizer.get_vocab()).difference(set(tokenizer_old.get_vocab()))
print(len(added_vocab))  # 13011

# loading and resizing the model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

# re-initializing the new embeddings
for t in tqdm(added_vocab):
    tt = tokenizer_old(t, add_special_tokens=False).input_ids
    if len(tt) == 0:
        tt = [tokenizer_old.unk_token_id]
    idx = tokenizer.convert_tokens_to_ids(t)
    model.model.shared.weight.data[idx] = model.model.shared.weight.data[tt].mean(0)

In [ ]:
added_token_id = tokenizer.convert_tokens_to_ids('olo_Latn')
similar_lang_id = tokenizer.convert_tokens_to_ids('fin_Latn')
print(model.model.shared.weight.data[added_token_id])
model.model.shared.weight.data[added_token_id] = model.model.shared.weight.data[similar_lang_id]
print(model.model.shared.weight.data[added_token_id])

# Train

In [9]:
# cleanup()
NLLB_NEW_PATH = '/kaggle/input/nllb-rus-kar/pytorch/dict/5'
model = AutoModelForSeq2SeqLM.from_pretrained(NLLB_NEW_PATH).cuda()
tokenizer = NllbTokenizer.from_pretrained(NLLB_NEW_PATH)

In [11]:
model.loss

AttributeError: 'M2M100ForConditionalGeneration' object has no attribute 'loss'

In [ ]:
LR = 1e-4

model.cuda();
optimizer = Adafactor(
    [p for p in model.parameters() if p.requires_grad],
    scale_parameter=False,
    relative_step=False,
    lr=LR,
    clip_threshold=1.0,
    weight_decay=1e-3,
)
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=1000)
scheduler1 = ConstantLR(optimizer, factor=1, total_iters=1000)
scheduler2 = ConstantLR(optimizer, factor=0.9, total_iters=8000)
scheduler3 = ConstantLR(optimizer, factor=0.8, total_iters=1000)
scheduler4 = ConstantLR(optimizer, factor=0.7, total_iters=11000)
scheduler = SequentialLR(optimizer, 
                         schedulers=[scheduler1, scheduler2, scheduler3, scheduler4], 
                         milestones=[1000,9000,10000])
# scheduler = SequentialLR(optimizer, 
#                          schedulers=[scheduler1, scheduler2, scheduler3, scheduler4], 
#                          milestones=[1000,9000,10000])
# scheduler = scheduler1

In [ ]:
LANGS = [('rus', 'rus_Cyrl'), ('kar', 'olo_Latn')]
# TODO переделать в dataset
def get_batch_pairs(batch_size, data=df_train):
    (l1, lang1), (l2, lang2) = random.sample(LANGS, 2) # здесь random чтобы переводила модель туда-сюда
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(preproc(item[l1]))
        yy.append(preproc(item[l2]))
    return xx, yy, lang1, lang2

print(get_batch_pairs(1))

In [ ]:
BATCH_SIZE = 8  # 32 already doesn't fit well to 15GB of GPU memory
MAX_LENGTH = 128 
TRAINING_STEPS = 21000

N_STEPS_TO_ESTIMATE = 100
losses = list()

preproc(df_dev.iloc[0]['kar'])

In [ ]:
def get_batched_validation(batch_size):
    (l1, lang1), (l2, lang2) = random.sample(LANGS, 2)
    for i in range(0, len(df_dev), batch_size):
        xx, yy = list(), list()
        sl = df_dev.iloc[i:i+batch_size]
        for _, row in sl.iterrows():
            xx.append(preproc(row[l1]))
            yy.append(preproc(row[l2]))
        yield xx, yy, lang1, lang2

In [ ]:
def validation(model, data=df_dev, batch_size=16):
    model.eval()
    av_loss = list()
    with torch.no_grad():
        for xx, yy, lang1, lang2 in get_batched_validation(batch_size):#range(len(data) // batch_size):
            # xx, yy, lang1, lang2 =  #get_batch_pairs(batch_size, data)
            tokenizer.src_lang = lang1
            x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH).to('cuda')
            tokenizer.src_lang = lang2
            y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH).to('cuda')
            y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100 # TODO: надо ли?
            loss = model(**x, labels=y.input_ids).loss
            av_loss.append(loss.item())
    model.train()
    return np.mean(av_loss)

In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
get_lr(optimizer)

In [ ]:
model.train()
x, y, loss = None, None, None
BEST_VAL_LOSS = 1e9
BEST_TRAIN_LOSS = 1e9
cleanup()


tq = trange(len(losses), TRAINING_STEPS)
for step in tq:
    xx, yy, lang1, lang2 = get_batch_pairs(BATCH_SIZE)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH).to(model.device)
        tokenizer.src_lang = lang2
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=MAX_LENGTH).to(model.device)
        # -100 is a magic value ignored in the loss function
        # because we don't want the model to learn to predict padding ids
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100

        loss = model(**x, labels=y.input_ids).loss
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

    except RuntimeError as e: 
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        cleanup()
        print('error', max(len(s) for s in xx + yy), e)
        continue
    if step % 1000 == 0:
        print(get_lr(optimizer))
    if step < 10000 and step % N_STEPS_TO_ESTIMATE == 0:
        # average for N steps
        train_loss = np.mean(losses[-N_STEPS_TO_ESTIMATE:])
        val_loss = validation(model)
        print(f'Train loss is {train_loss} and valid loss is {val_loss} at step {step}')
        if train_loss < 1.75 and train_loss < BEST_TRAIN_LOSS and val_loss < BEST_VAL_LOSS:
            BEST_TRAIN_LOSS = train_loss
            BEST_VAL_LOSS = val_loss
            model.save_pretrained(MODEL_SAVE_PATH)
            tokenizer.save_pretrained(MODEL_SAVE_PATH)
            print(f'model_saved')
    elif step >= 10000 and step % 300 == 0:
        train_loss = np.mean(losses[-300:])
        val_loss = validation(model)
        print(f'Train loss is {train_loss} and valid loss is {val_loss} at step {step}')
        if train_loss < 1.75 and train_loss < BEST_TRAIN_LOSS and val_loss < BEST_VAL_LOSS:
            BEST_TRAIN_LOSS = train_loss
            BEST_VAL_LOSS = val_loss
            model.save_pretrained(MODEL_SAVE_PATH)
            tokenizer.save_pretrained(MODEL_SAVE_PATH)
            print(f'model_saved')